In [1]:
import pyes
import json

def pp(o): 
    print (json.dumps(o, indent=1))

host = '127.0.0.1:9200'

fid1 = '139624776093433'
fid2 = '180080052032125'

def aa(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(tagg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    print (json.dumps(result.aggs,indent=2)) 
    

aa(fid1)
aa(fid2)


pyes object <pyes.es.ES object at 0x10c7a5eb8> is being destroyed, but bulk operations have not been flushed. Call force_bulk()!


{
  "pid": {
    "doc_count_error_upper_bound": 10,
    "sum_other_doc_count": 82582,
    "buckets": [
      {
        "doc_count": 7574,
        "key": "203507002993446_952427894768016"
      },
      {
        "doc_count": 6273,
        "key": "203507002993446_1150921904918613"
      },
      {
        "doc_count": 6172,
        "key": "203507002993446_902107496466723"
      },
      {
        "doc_count": 5892,
        "key": "203507002993446_964119643598841"
      },
      {
        "doc_count": 5766,
        "key": "203507002993446_871353986208741"
      },
      {
        "doc_count": 5534,
        "key": "203507002993446_982225088454963"
      },
      {
        "doc_count": 5509,
        "key": "203507002993446_972663902744415"
      },
      {
        "doc_count": 5352,
        "key": "203507002993446_906953715982101"
      },
      {
        "doc_count": 5339,
        "key": "203507002993446_1035838583093613"
      },
      {
        "doc_count": 5185,
        "key": "2035070

In [2]:
def bb(fid):
    conn = pyes.es.ES(host)
    tq = pyes.query.TermQuery(field="fid", value=fid)
    #tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg = pyes.aggs.DateHistogramAgg('month' ,field='created_time', interval='month',sub_aggs=[])
    tagg = pyes.aggs.TermsAgg('pid', field= 'fid_pid')
    DHAgg.sub_aggs.append(tagg) 
    qsearch = pyes.query.Search(tq) 
    qsearch.agg.add(DHAgg) 
    result = conn.search(query=qsearch , indices='facebook_nested' , doc_types='comment')
    print (json.dumps(result.aggs,indent=2) )
bb(fid2)

pyes object <pyes.es.ES object at 0x10c79c7b8> is being destroyed, but bulk operations have not been flushed. Call force_bulk()!


{
  "month": {
    "buckets": [
      {
        "pid": {
          "doc_count_error_upper_bound": 0,
          "sum_other_doc_count": 0,
          "buckets": [
            {
              "doc_count": 3,
              "key": "134004310003557_425747687495883"
            }
          ]
        },
        "doc_count": 3,
        "key": 1354320000000,
        "key_as_string": "2012-12-01T00:00:00.000Z"
      },
      {
        "pid": {
          "doc_count_error_upper_bound": 0,
          "sum_other_doc_count": 0,
          "buckets": [
            {
              "doc_count": 60,
              "key": "134004310003557_433302006740451"
            },
            {
              "doc_count": 29,
              "key": "134004310003557_429439150460070"
            },
            {
              "doc_count": 25,
              "key": "134004310003557_436957886374863"
            },
            {
              "doc_count": 24,
              "key": "134004310003557_431164023620916"
            },
 

In [3]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, show, output_file

# generate some synthetic time series for six different categories
cats = list("abcdef")
yy = np.random.randn(2000)
g = np.random.choice(cats, 2000)
for i, l in enumerate(cats):
    yy[g == l] += i // 2
df = pd.DataFrame(dict(score=yy, group=g))

# find the quartiles and IQR for each category
groups = df.groupby('group')
q1 = groups.quantile(q=0.25)
q2 = groups.quantile(q=0.5)
q3 = groups.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5*iqr
lower = q1 - 1.5*iqr

# find the outliers for each category
def outliers(group):
    cat = group.name
    return group[(group.score > upper.loc[cat]['score']) | (group.score < lower.loc[cat]['score'])]['score']
out = groups.apply(outliers).dropna()

# prepare outlier data for plotting, we need coordinates for every outlier.
if not out.empty:
    outx = []
    outy = []
    for cat in cats:
        # only add outliers if they exist
        if not out.loc[cat].empty:
            for value in out[cat]:
                outx.append(cat)
                outy.append(value)

p = figure(tools="save", background_fill_color="#EFE8E2", title="", x_range=cats)

# if no outliers, shrink lengths of stems to be no longer than the minimums or maximums
qmin = groups.quantile(q=0.00)
qmax = groups.quantile(q=1.00)
upper.score = [min([x,y]) for (x,y) in zip(list(qmax.loc[:,'score']),upper.score)]
lower.score = [max([x,y]) for (x,y) in zip(list(qmin.loc[:,'score']),lower.score)]

# stems
p.segment(cats, upper.score, cats, q3.score, line_width=2, line_color="black")
p.segment(cats, lower.score, cats, q1.score, line_width=2, line_color="black")

# boxes
p.rect(cats, (q3.score+q2.score)/2, 0.7, q3.score-q2.score,
    fill_color="#E08E79", line_width=2, line_color="black")
p.rect(cats, (q2.score+q1.score)/2, 0.7, q2.score-q1.score,
    fill_color="#3B8686", line_width=2, line_color="black")

# whiskers (almost-0 height rects simpler than segments)
p.rect(cats, lower.score, 0.2, 0.01, line_color="black")
p.rect(cats, upper.score, 0.2, 0.01, line_color="black")

# outliers
if not out.empty:
    p.circle(outx, outy, size=6, color="#F38630", fill_alpha=0.6)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = "white"
p.grid.grid_line_width = 2
p.xaxis.major_label_text_font_size="12pt"

output_file("boxplot.html", title="boxplot.py example")

show(p)

In [3]:
out

group      
b      523    -2.950132
       667     2.972560
       834     2.930676
       946    -3.607122
c      135     3.801574
       1419   -1.965743
d      374    -2.662281
       1109    4.273440
       1381    3.699180
       1750   -1.715891
       1893    3.554335
       1902    4.088982
e      1193    4.738827
       1496   -1.083148
       1646   -0.632557
f      62      5.221453
       66     -0.652118
       1801    4.637968
Name: score, dtype: float64

In [4]:
groups

In [5]:
df

,group,score
0,c,0.569101
1,f,3.995808
2,e,0.546930
3,b,0.238516
4,f,-0.348212
5,d,1.459796
6,b,0.457813
7,c,0.785527
8,e,1.012897
9,b,0.793354
